<a href="https://colab.research.google.com/github/fluidboss/CFD_Course/blob/main/MD_LAMMPS_prebuild_using_conda_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RUNNING LAMMPS ON GOOGLE COLAB WITH GPU

Tutorial for MECH0074 - Week 2, Lecture 3, in class Activity

The purpose of this activity is to explore the molecular view of the world and try to relate these results to the macroscopic view of the world.

This code will run a Laamps code.

Mount drive of google colab




##Installing LAMMPS

Check System Specification

In [29]:
%%bash
#Default for free account: 1GPU and 2 CPU with 1TB HDD and 12GB RAM
lscpu
nvidia-smi

Architecture:                            x86_64
CPU op-mode(s):                          32-bit, 64-bit
Address sizes:                           46 bits physical, 48 bits virtual
Byte Order:                              Little Endian
CPU(s):                                  2
On-line CPU(s) list:                     0,1
Vendor ID:                               GenuineIntel
Model name:                              Intel(R) Xeon(R) CPU @ 2.20GHz
CPU family:                              6
Model:                                   79
Thread(s) per core:                      2
Core(s) per socket:                      1
Socket(s):                               1
Stepping:                                0
BogoMIPS:                                4399.99
Flags:                                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda install -c conda-forge lammps -y

Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - lammps


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    attr-2.5.2                 |       h39aace5_0          66 KB  conda-forge
    binutils_impl_linux-64-2.43|       h4bf12b8_2         5.4 MB  conda-forge
    binutils_linux-64-2.43     |       h4852527_2          34 KB  conda-forge
    blosc-1.21.6               |       he440d0b_1          47 KB  conda-forge
    ca-certificates-2025.8.3   |       hbd8a1cb_0         151 KB  conda-forge
    certifi-2025.8.3           |     pyhd8ed1ab_0         155 KB  conda-forge
    cmake-4.0.0                |       h74e3db0_0        19.5 MB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    fftw-3.3.10                |mpi_openmpi_h99e6

In [ ]:
!lmp -h


Large-scale Atomic/Molecular Massively Parallel Simulator - 29 Aug 2024

Usage example: lmp -var t 300 -echo screen -in in.alloy

List of command line options supported by this LAMMPS executable:

-echo none/screen/log/both  : echoing of input script (-e)
-help                       : print this help message (-h)
-in none/filename           : read input from file or stdin (default) (-i)
-kokkos on/off ...          : turn KOKKOS mode on or off (-k)
-log none/filename          : where to send log output (-l)
-mdi '<mdi flags>'          : pass flags to the MolSSI Driver Interface
-mpicolor color             : which exe in a multi-exe mpirun cmd (-m)
-cite                       : select citation reminder style (-c)
-nocite                     : disable citation reminder (-nc)
-nonbuf                     : disable screen/logfile buffering (-nb)
-package style ...          : invoke package command (-pk)
-partition size1 size2 ...  : assign partition sizes (-p)
-plog basename              : 

Aluminum example
from
https://cavs.msstate.edu/icme/code/lammps/tutorials/lammps/uniaxial_tension.php

In [36]:
%%writefile in.aluminum


# ------------------------ INITIALIZATION ----------------------------
units 		metal
dimension	3
boundary	p	p	p
atom_style	atomic
variable latparam equal 4.05

# ----------------------- ATOM DEFINITION ----------------------------
lattice		fcc ${latparam}
region		whole block 0 10 0 10 0 10
create_box	1 whole
lattice 	fcc ${latparam} orient x 1 0 0 orient y 0 1 0 orient z 0 0 1
create_atoms	1 region whole

# ------------------------ FORCE FIELDS ------------------------------
pair_style	eam/alloy
pair_coeff	* * Al99.eam.alloy Al

# ------------------------- SETTINGS ---------------------------------
compute csym all centro/atom fcc
compute peratom all pe/atom

######################################
# EQUILIBRATION
reset_timestep	0
timestep 0.001
velocity all create 300 12345 mom yes rot no
fix 1 all npt temp 300 300 1 iso 0 0 1 drag 1

# Set thermo output
thermo 1000
thermo_style custom step lx ly lz press pxx pyy pzz pe temp

# Run for at least 10 picosecond (assuming 1 fs timestep)
run 20000
unfix 1

# Store final cell length for strain calculations
variable tmp equal "lx"
variable L0 equal ${tmp}
print "Initial Length, L0: ${L0}"

######################################
# DEFORMATION
reset_timestep	0

fix		1 all npt temp 300 300 1 y 0 0 1 z 0 0 1 drag 1
variable srate equal 1.0e10
variable srate1 equal "v_srate / 1.0e12"
fix		2 all deform 1 x erate ${srate1} units box remap x

# Output strain and stress info to file
# for units metal, pressure is in [bars] = 100 [kPa] = 1/10000 [GPa]
# p2, p3, p4 are in GPa
variable strain equal "(lx - v_L0)/v_L0"
variable p1 equal "v_strain"
variable p2 equal "-pxx/10000"
variable p3 equal "-pyy/10000"
variable p4 equal "-pzz/10000"
fix def1 all print 100 "${p1} ${p2} ${p3} ${p4}" file Al_SC_100.def1.txt screen no

# Use cfg for AtomEye
dump 		1 all cfg 250 dump.tensile_*.cfg mass type xs ys zs c_csym c_peratom fx fy fz
dump_modify 1 element Al

# Display thermo
thermo 	1000
thermo_style	custom step v_strain temp v_p2 v_p3 v_p4 ke pe press

run		20000

######################################
# SIMULATION DONE
print "All done"

Overwriting in.aluminum


In [37]:
!mpirun --allow-run-as-root -n 1 lmp -in in.aluminum

LAMMPS (29 Aug 2024)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 4.05 4.05 4.05
Created orthogonal box = (0 0 0) to (40.5 40.5 40.5)
  1 by 1 by 1 MPI processor grid
Lattice spacing in x,y,z = 4.05 4.05 4.05
Created 4000 atoms
  using lattice units in orthogonal box = (0 0 0) to (40.5 40.5 40.5)
  create_atoms CPU = 0.001 seconds
ERROR on proc 0: cannot open eam/alloy potential file Al99.eam.alloy: No such file or directory (src/potential_file_reader.cpp:58)
Last command: pair_coeff	* * Al99.eam.alloy Al
--------------------------------------------------------------------------
MPI_ABORT was invoked on rank 0 in communicator MPI_COMM_WORLD
  Proc: [[33367,1],0]
  Errorcode: 1

NOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes.
You may or may not see output from other processes, depending on
exactly when Open MPI kills them.
------------------------------------------

##Run your lammps script

In [32]:
%%writefile in.crack

# 2d LJ crack simulation

dimension	2
boundary	s s p

atom_style	atomic
neighbor	0.3 bin
neigh_modify	delay 5

# create geometry

lattice		hex 0.93
region		box block 0 100 0 40 -0.25 0.25
create_box	5 box
create_atoms	1 box

mass		1 1.0
mass		2 1.0
mass		3 1.0
mass		4 1.0
mass		5 1.0

# LJ potentials

pair_style	lj/cut 2.5
pair_coeff	* * 1.0 1.0 2.5

# define groups

region	        1 block INF INF INF 1.25 INF INF
group		lower region 1
region		2 block INF INF 38.75 INF INF INF
group		upper region 2
group		boundary union lower upper
group		mobile subtract all boundary

region		leftupper block INF 20 20 INF INF INF
region		leftlower block INF 20 INF 20 INF INF
group		leftupper region leftupper
group		leftlower region leftlower

set		group leftupper type 2
set		group leftlower type 3
set		group lower type 4
set		group upper type 5

# initial velocities

compute	  	new mobile temp
velocity	mobile create 0.01 887723 temp new
velocity	upper set 0.0 0.3 0.0
velocity	mobile ramp vy 0.0 0.3 y 1.25 38.75 sum yes

# fixes

fix		1 all nve
fix		2 boundary setforce NULL 0.0 0.0

# run

timestep	0.003
thermo		200
thermo_modify	temp new

neigh_modify	exclude type 2 3

dump		1 all atom 500 dump.crack

#dump		2 all image 250 image.*.jpg type type &
#		zoom 1.6 adiam 1.5
#dump_modify	2 pad 4

#dump		3 all movie 250 movie.mpg type type &
#		zoom 1.6 adiam 1.5
#dump_modify	3 pad 4

run		5000

Overwriting in.crack


In [33]:
!mpirun --allow-run-as-root -n 1 lmp -in in.crack

LAMMPS (29 Aug 2024)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 1.1142771 1.9299846 1.1142771
Created orthogonal box = (0 0 -0.27856928) to (111.42771 77.199383 0.27856928)
  1 by 1 by 1 MPI processor grid
Created 8141 atoms
  using lattice units in orthogonal box = (-0.011142771 -0.0077199383 -0.27856928) to (111.43885 77.207103 0.27856928)
  create_atoms CPU = 0.001 seconds
302 atoms in group lower
302 atoms in group upper
604 atoms in group boundary
7537 atoms in group mobile
841 atoms in group leftupper
841 atoms in group leftlower
Setting atom values ...
  841 settings made for type
Setting atom values ...
  841 settings made for type
Setting atom values ...
  302 settings made for type
Setting atom values ...
  302 settings made for type
Generated 0 of 10 mixed pair_coeff terms from geometric mixing rule
Neighbor list info ...
  update: every = 1 steps, delay = 5 steps, check

In [ ]:
!ls -lh

total 2.6M
-rw-r--r-- 1 root root  21K Sep 30 13:08 condacolab_install.log
-rw-r--r-- 1 root root 2.5M Sep 30 13:36 dump.crack
-rw-r--r-- 1 root root 1.4K Sep 30 13:36 in.crack
-rw-r--r-- 1 root root 6.7K Sep 30 13:36 log.lammps
-rw-r--r-- 1 root root 2.8K Sep 30 13:36 results.zip
drwxr-xr-x 1 root root 4.0K Sep 26 13:40 sample_data


In [ ]:
from google.colab import files
!zip results.zip dump.*
files.download("results.zip")

  adding: dump.crack (deflated 70%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install ovito
from ovito.io import import_file
from ovito.vis import Viewport

pipeline = import_file("crack.lammpstrj")
vp = Viewport()
vp.type = Viewport.Type.Perspective
vp.camera_pos = (0,0,50)
vp.fov = 60.0
vp.render_image(size=(600,600), filename="frame.png")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 554.8/554.8 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.2/167.2 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 30.4 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'ovito'